In [414]:
import numpy as np
import matplotlib.pyplot as plt
import scikitplot as skplt
from imblearn.over_sampling import SMOTE

import pandas as pd

from sklearn.preprocessing import StandardScaler # stanardization
from sklearn.preprocessing import LabelEncoder # Label --> Number
from sklearn.preprocessing import minmax_scale
import seaborn as sns
from sklearn.model_selection import train_test_split # Training/Test split
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error


from sklearn import datasets, decomposition

from sklearn.manifold import MDS

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB

from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm

from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix, classification_report # Confusion Matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.metrics import roc_curve

from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score, KFold

from sklearn.decomposition import PCA

from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample

from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier

In [475]:
project_df = pd.read_csv('stat473_train.csv')
train_data, test_data = train_test_split(project_df, test_size=0.20, random_state=1)

y_train = train_data['Target'] # binary variable

#X_train = train_data[['A','C', 'D', 'E', 'F', 'H', 'I', 'J', 'L', 'M', 'N', 'O', 'Q', 'R', 'S', 'T']] 
#full
X_train = train_data[['A','B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T']] 
X_train = X_train.drop(['B', 'G', 'K', 'O', 'P', 'R'], axis=1)



y_test = test_data['Target'] # binary variable
#full
#X_test = test_data[['A','B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T']] 
#X_test = X_test.drop(['B', 'G', 'K', 'P'], axis=1)
#delete F
#X_test = X_test.drop(['F'], axis=1)

# Save the new datasets as CSV files
train_data.to_csv('train1.csv', index=False)
test_data.to_csv('test1.csv', index=False)

project_train_df = pd.read_csv('train1.csv')
project_test_df = pd.read_csv('test1.csv')
#y_test = project_test_df['Target'] # binary variable
X_test = project_test_df[['A','B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T']] 
X_test = X_test.drop(['B', 'G', 'K', 'O', 'P', 'R'], axis=1)

#print(project_df)
#print(project_train_df)
#y = project_train_df['Target'] # binary variable
#X = project_train_df[['A','B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T']] 

#y = project_train_df['Target'] # binary variable
#X = project_train_df[['A','C', 'D', 'E', 'F', 'H', 'I', 'J', 'L', 'M', 'N', 'O', 'Q', 'R', 'S', 'T']] 

In [424]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.10, random_state=1 ,stratify=y)

In [476]:
#Run this cell when you want to apply SMOTE on test?
oversample = SMOTE(random_state=1)
X_train, y_train = oversample.fit_resample(X_train, y_train)
y_train.value_counts()

Target
0    5218
1    5218
Name: count, dtype: int64

In [302]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Get importance
feature_importances = model.feature_importances_

# Convert to a DataFrame for easier handling

features_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importances
})

# Sort features by importance
features_df.sort_values(by='Importance', ascending=False, inplace=True)

# Select top N features, for example top 10
top_features = features_df.head(20)['Feature'].values
X_reduced = X_train[top_features]
print(X_reduced)

              C         Q        A          M        T         D         N  \
2721   9.089100  -7.33140  11.2000  -8.204100  -7.0701   9.48850  11.66900   
2616   7.130200 -17.39300  -5.3012  -6.704900   1.0123 -25.20800  -9.94820   
4068   2.712800  -0.25422 -15.3630 -14.601000  -8.8995   0.99666   8.05920   
5102 -23.220000  13.94100   9.4726   9.314000  13.9690  -8.93410   2.47370   
792   17.908000  -5.03960  27.6610  -0.945750   7.0839  13.16800  21.52500   
...         ...       ...      ...        ...      ...       ...       ...   
417   -0.064823   6.31890  -1.3495   8.958400   3.4652  -3.46520  -2.11790   
6188  -3.273600  -5.78820 -11.0160  19.727000  -2.9987   1.59430 -10.01700   
3845  -8.193500   5.99220   5.7835  -5.164800   6.7328 -10.91800  12.18700   
770    3.268900 -16.42100  -6.4509  -2.949700 -12.9140   3.27680   0.89487   
1933   5.317400   3.61570  14.2870  -0.037809  -6.6960  -0.14706  11.94700   

            E        J        S        H         L        I    

In [210]:
# Randomly sample 1000 instances from the reduced feature set
X_sampled = resample(X_reduced, n_samples=1000, random_state=0, replace=False)

In [219]:
pca = decomposition.PCA(n_components='mle')
pca.fit(X_sampled)
PC = pca.transform(X_sampled)
pca_loadings = pca.components_.T
print(PC[:, :])

[[-9.09724725e+00]
 [ 1.05480913e+01]
 [-3.40074539e+00]
 [ 4.28939974e+00]
 [-1.93938521e+00]
 [-3.48908479e+00]
 [-2.31836117e+01]
 [-1.47907469e+01]
 [ 1.40118079e+00]
 [-4.22145788e+00]
 [ 4.52911266e+00]
 [ 8.98872101e+00]
 [-3.15184778e+00]
 [ 1.72252590e+01]
 [-1.26473179e+01]
 [-1.56931568e+01]
 [ 3.70970700e+00]
 [-4.44408005e+00]
 [ 2.71152200e+00]
 [ 1.17629019e+01]
 [ 3.19358027e+00]
 [-1.10784970e+01]
 [-3.39533470e-02]
 [-1.31626636e+01]
 [ 3.48895042e+00]
 [ 9.42359297e+00]
 [ 2.74261488e+01]
 [ 1.62089932e+01]
 [ 1.48525040e+01]
 [-5.24991863e+00]
 [ 8.64493399e+00]
 [ 1.57145445e+01]
 [ 2.07522701e+00]
 [-2.72203373e+00]
 [ 2.47772680e+00]
 [ 6.94666207e+00]
 [ 3.51965481e+00]
 [-1.53111531e+01]
 [-1.74139287e+01]
 [-4.53015098e+00]
 [-9.31372579e+00]
 [ 1.70020606e+00]
 [-1.92656751e+01]
 [ 1.82042599e+01]
 [ 1.97409835e+01]
 [-1.14008263e+01]
 [ 1.57567789e+01]
 [ 2.14115725e+00]
 [-2.08569150e+00]
 [ 2.66411435e+00]
 [-1.10017867e+01]
 [-8.56088390e+00]
 [-2.1115904

In [212]:
mds = MDS(n_components=2, random_state=0)
MD = mds.fit_transform(X_sampled)
print(MD[:,:2])

[[ 25.02572321  -8.63364816]
 [-50.52050901  -3.23431348]
 [ -9.30340481 -52.3456249 ]
 ...
 [-53.96953732 -27.37460089]
 [-23.46498086   4.78227924]
 [-40.83867171 -22.13286547]]


In [25]:
pca = decomposition.PCA(n_components=2)
pca.fit(X)
PC = pca.transform(X)

In [513]:
xgb_model = XGBClassifier()

# Define the parameter grid
param_grid = {
    'n_estimators': [845, 850, 855],
    'learning_rate': [0.3, 0.35, 0.4],
    'max_depth': [3, 4, 5]
}


In [514]:
f1_scorer = make_scorer(f1_score, average='macro')  # Use 'binary' for binary classification

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring=f1_scorer, cv=5, verbose=3)

In [515]:
# Fit GridSearchCV
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV 1/5] END learning_rate=0.3, max_depth=3, n_estimators=845;, score=0.951 total time=   0.8s
[CV 2/5] END learning_rate=0.3, max_depth=3, n_estimators=845;, score=0.957 total time=   1.4s
[CV 3/5] END learning_rate=0.3, max_depth=3, n_estimators=845;, score=0.956 total time=   1.3s
[CV 4/5] END learning_rate=0.3, max_depth=3, n_estimators=845;, score=0.962 total time=   1.0s
[CV 5/5] END learning_rate=0.3, max_depth=3, n_estimators=845;, score=0.954 total time=   1.6s
[CV 1/5] END learning_rate=0.3, max_depth=3, n_estimators=850;, score=0.949 total time=   1.1s
[CV 2/5] END learning_rate=0.3, max_depth=3, n_estimators=850;, score=0.958 total time=   1.3s
[CV 3/5] END learning_rate=0.3, max_depth=3, n_estimators=850;, score=0.956 total time=   1.2s
[CV 4/5] END learning_rate=0.3, max_depth=3, n_estimators=850;, score=0.960 total time=   1.0s
[CV 5/5] END learning_rate=0.3, max_depth=3, n_estimators=850;, score=0.954 total t

[CV 2/5] END learning_rate=0.35, max_depth=5, n_estimators=855;, score=0.955 total time=   1.4s
[CV 3/5] END learning_rate=0.35, max_depth=5, n_estimators=855;, score=0.956 total time=   1.3s
[CV 4/5] END learning_rate=0.35, max_depth=5, n_estimators=855;, score=0.966 total time=   1.3s
[CV 5/5] END learning_rate=0.35, max_depth=5, n_estimators=855;, score=0.957 total time=   1.4s
[CV 1/5] END learning_rate=0.4, max_depth=3, n_estimators=845;, score=0.949 total time=   1.1s
[CV 2/5] END learning_rate=0.4, max_depth=3, n_estimators=845;, score=0.954 total time=   0.9s
[CV 3/5] END learning_rate=0.4, max_depth=3, n_estimators=845;, score=0.955 total time=   1.2s
[CV 4/5] END learning_rate=0.4, max_depth=3, n_estimators=845;, score=0.958 total time=   2.0s
[CV 5/5] END learning_rate=0.4, max_depth=3, n_estimators=845;, score=0.956 total time=   1.7s
[CV 1/5] END learning_rate=0.4, max_depth=3, n_estimators=850;, score=0.951 total time=   2.0s
[CV 2/5] END learning_rate=0.4, max_depth=3, n

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             param_grid={'learning_rate': [0.3, 0.35, 0.4],
                         'max_depth': [3, 4, 5],
                         'n_estimators': [845, 850, 855]},
             scoring=make_scorer(f1_score, response_method='predict', average=macro),
             verbose=3)

In [516]:
# Best parameters and best score
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation F1 score: {:.2f}".format(grid_search.best_score_))

# Optional: Evaluate on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Test F1 Score: ", f1_score(y_test, y_pred, average='macro'))  # or 'binary'

Best parameters: {'learning_rate': 0.3, 'max_depth': 5, 'n_estimators': 850}
Best cross-validation F1 score: 0.96
Test F1 Score:  0.8806681803648571


In [368]:
# Define the cross-validation configuration
kf = KFold(n_splits=10, shuffle=True, random_state=42)  # 10-fold cross-validation

# Define your F1 scorer, ensuring it matches the needs of your classification (binary, multi-class, etc.)
f1_scorer = make_scorer(f1_score, average='macro')  # Adjust 'macro' as needed

# Perform cross-validation
cv_scores = cross_val_score(best_model, X_train, y_train, cv=kf, scoring=f1_scorer)

# Print out the mean and standard deviation of the cross-validation scores
print("CV Mean F1 Score: {:.2f}".format(cv_scores.mean()))
print("CV Standard Deviation in F1 Score: {:.2f}".format(cv_scores.std()))

CV Mean F1 Score: 0.85
CV Standard Deviation in F1 Score: 0.02


SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (515130869.py, line 1)

In [553]:
XGBM = XGBClassifier(n_estimators= 838, learning_rate = 0.3, max_depth = 3)
XGBM.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.3, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=838, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [554]:
y_pred = XGBM.predict(X_test)
confmat = pd.DataFrame(confusion_matrix(y_test, y_pred),
                      index=['True[0]','True[1]'],
                      columns=['Predict[0]', 'Predict[1]'])
confmat

,Predict[0],Predict[1]
True[0],1250,60
True[1],35,255


In [555]:
print(classification_report(y_test, y_pred))
print('F1: %.3f' % f1_score(y_true=y_test, y_pred=y_pred))
#default f1: 0.775
# results on test
#w/o SMOTE n_estimators=700, learning_rate = 0.3, max_depth = 3 , F1: 0.788
#w/ SMOTE n_estimators=700, learning_rate = 0.3, max_depth = 3 , F1: 0.803
#w/o SMOTE n_estimators=800, learning_rate = 0.3, max_depth = 3 , F1: 0.798
#w/ SMOTE n_estimators=700, learning_rate = 0.3, max_depth = 3 , F1: 0.809
#w/ SMOTE n_estimators=800, learning_rate = 0.3, max_depth = 3 , F1: 0.832, with removal of input features
# subsetted dataset, 'learning_rate': 0.3, 'max_depth': 4, 'n_estimators': 900, F1: 0.813
# subsetted dataset: n_estimators=900, learning_rate = 0.3, max_depth = 4, F1 score: 0.829
#F1: 0.838, subsetted dataset : n_estimators= 855, learning_rate = 0.3, max_depth = 3

              precision    recall  f1-score   support

           0       0.97      0.95      0.96      1310
           1       0.81      0.88      0.84       290

    accuracy                           0.94      1600
   macro avg       0.89      0.92      0.90      1600
weighted avg       0.94      0.94      0.94      1600

F1: 0.843


In [534]:
df_pca = pd.DataFrame(pca_loadings, columns=['PC1', 'PC2', 'PC3'], index=X_sampled.columns)

# Create a DataFrame for MDS coordinates
df_mds = pd.DataFrame(MD, columns=['MDS1', 'MDS2','MDS3'])

# Merge the two DataFrames based on index for PCA (which corresponds to feature names)
# For simplicity, I'm assuming you want to relate PCA and MDS results by feature name index.
# Note: This step might need adjustments based on how exactly you wish to relate PCA and MDS results.
df_combined = df_pca.reset_index().rename(columns={'index': 'Features'})
df_combined['MDS1'] = df_mds['MDS1']
df_combined['MDS2'] = df_mds['MDS2']
df_combined['MDS3'] = df_mds['MDS3']

# Optionally, display the DataFrame
print(df_combined)

ValueError: Shape of passed values is (20, 1), indices imply (20, 3)